#  Reduced Events Table

__Description__: Reduced excess rainfall is calculated using a randomly generated stormwater removal rate and capacity for each event.

__Input__: A JSON file containing the incremental excess rainfall for a suite of events which may have different durations and boundary condition names. 

__Output__: 

- The incremental reduced excess rainfall for each event as a JSON.

- The incremental stormwater amount for each event as a JSON.

- A metadata file containing the stormwater removal rate, the stormwater capacity, and the seed of the random number generator.

---

## A. Load Libraries, Parameters, and Data:

### Libraries:

In [1]:
import sys
sys.path.append('core')
import hydromet_reduced
from hydromet import*

### Parameters: 

#### Papermill (site specific):

In [2]:
### Minimum and maximum storm water removal rates:
minrate = 0.0459 # [L^3/30min]
maxrate = 0.2375 # [L^3/30min]

## Filenames and paths:
filename = 'DC_P01.json'
root_dir = pl.Path(os.getcwd())
outputs_dir = root_dir/'Outputs'

## Options:
seed = np.random.randint(low=0, high=10000)
display_print = True

##### Convert all paths to objects:

In [3]:
outputs_dir = pl.Path(outputs_dir)

### Load the excess rainfall data:

In [4]:
with open(outputs_dir/filename) as f:
    EventsTable = json.load(f)

##### Extract the durations:

In [5]:
durations = list(EventsTable.keys())
print('Durations:', durations)

Durations: ['H06', 'H12', 'H24', 'H96']


##### Extract the boundary condition names:

In [6]:
BCN = list(EventsTable[durations[0]]['BCName'].keys())
print('Boundary Condition Names:', BCN)

Boundary Condition Names: ['D01']


---

## B. Calculate Reduced Excess Rainfall:

In [7]:
ReducedTable = {}
StormwaterTable = {}
SW_variables = {}

for dur in durations:
    dic_dur = EventsTable[dur]
    time_ord = dic_dur['time_idx_ordinate']
    time_idx = dic_dur['time_idx']
    print('Duration:', dur)
    ts = determine_timestep(dic_dur, display_print)
    adj_rate, maximum_capacity, seed = storm_water_simulator(minrate, maxrate, ts, seed, display_print)
    dic_BCN = {}
    dic_BCN_SW = {}
    
    for name in BCN:
        dic_events = dic_dur['BCName'][name]
        dic_reduced = {}
        dic_stormwater = {}
        
        for event in dic_events.keys():
            unreduced = dic_events[event]
            reduced = calculate_reduced_excess(unreduced, adj_rate, maximum_capacity)
            dic_reduced[event] = reduced
            dic_stormwater[event] = list(np.array(unreduced)-np.array(reduced))
            
        dic_BCN[name] = dic_reduced
        dic_BCN_SW[name] = dic_stormwater
        
    ReducedTable[dur] = {'time_idx_ordinate':time_ord,'time_idx':time_idx, 'BCName': dic_BCN}
    StormwaterTable[dur] = {'time_idx_ordinate':time_ord,'time_idx':time_idx, 'BCName': dic_BCN_SW}
    SW_variables[dur] = {'Rate': adj_rate, 'Capacity': maximum_capacity, 'Seed':seed}

Duration: H06
Time Step: 0.5 Hours
Rate: 0.1753701392104806 Maximum Capacity: 8.41776668210307 Seed: 51
Duration: H12
Time Step: 1.0 Hours
Rate: 0.3507402784209612 Maximum Capacity: 8.41776668210307 Seed: 51
Duration: H24
Time Step: 2.0 Hours
Rate: 0.7014805568419225 Maximum Capacity: 8.41776668210307 Seed: 51
Duration: H96
Time Step: 4.0 Hours
Rate: 1.402961113683845 Maximum Capacity: 8.41776668210307 Seed: 51


---

## C. Plot the Results:

In [9]:
events = ReducedTable[durations[0]]['BCName'][BCN[0]].keys()

dict_keys(['E0001', 'E0002', 'E0003', 'E0004', 'E0005', 'E0006', 'E0007', 'E0008', 'E0009', 'E0010', 'E0011', 'E0012', 'E0013', 'E0014', 'E0015', 'E0016', 'E0017', 'E0018', 'E0019', 'E0020', 'E0021', 'E0022', 'E0023', 'E0024', 'E0025', 'E0026', 'E0027', 'E0028', 'E0029', 'E0030', 'E0031', 'E0032', 'E0033', 'E0034', 'E0035', 'E0036', 'E0037', 'E0038', 'E0039', 'E0040', 'E0041', 'E0042', 'E0043', 'E0044', 'E0045', 'E0046', 'E0047', 'E0048', 'E0049', 'E0050', 'E0051', 'E0052', 'E0053', 'E0054', 'E0055', 'E0056', 'E0057', 'E0058', 'E0059', 'E0060', 'E0061', 'E0062', 'E0063', 'E0064', 'E0065', 'E0066', 'E0067', 'E0068', 'E0069', 'E0070', 'E0071', 'E0072', 'E0073', 'E0074', 'E0075', 'E0076', 'E0077', 'E0078', 'E0079', 'E0080', 'E0081', 'E0082', 'E0083', 'E0084', 'E0085', 'E0086', 'E0087', 'E0088', 'E0089', 'E0090', 'E0091', 'E0092', 'E0093', 'E0094', 'E0095', 'E0096', 'E0097', 'E0098', 'E0099', 'E0100', 'E0101', 'E0102', 'E0103', 'E0104', 'E0105', 'E0106', 'E0107', 'E0108', 'E0109', 'E0110',

In [ ]:
def plot_reduced_excess(df: pd.DataFrame, df2: pd.DataFrame, col: pd.Series, tstep: int, duration: int, adjustment_rate: float, max_capacity: float) -> plt.subplots:
    '''
    '''
    fig, ax = plt.subplots(figsize=(24,3))
    xmax=int(duration*24*(60/tstep))
    ax.plot(df[0:xmax].index, df[0:xmax][col], '--', color='grey', 
                                    label = 'Incremental Excess Rainfall')
    ax.set_ylabel('Inches')
    ax.set_xlabel('Hours')
    ax.plot(df2[0:xmax].index, df2[0:xmax][col], '-', color = 'blue', 
        label = 'Adjusted Excess Rainfall:\n{} inches/hour'.format(round(adjustment_rate, 5)))
    ax.set_title('Cumulative Excess Rainfall {} inches\n Design Capacity {} inches'.format(np.round(df[col].sum(),2),np.round(max_capacity,2)))
    ax.grid()
    ax.legend()

In [ ]:
plot_reduced_excess(df, df2, col, tstep, duration, adjustment_rate, max_capacity)

---

## D. Save the Results:

### Reduced excess rainfall:

In [ ]:
with open(outputs_dir/'Reduced_{0}'.format(filename), 'w') as f:
    json.dump(ReducedTable, f)

### Stormwater:

In [ ]:
with open(outputs_dir/'Stormwater_{0}'.format(filename), 'w') as f:
    json.dump(StormwaterTable, f)

### Metadata:

In [ ]:
with open(outputs_dir/'SWMetadata_{0}'.format(filename), 'w') as f:
    json.dump(SW_variables, f)

In [ ]:
SW_variables

---

## END